### Import required packages

In [1]:
import pandas as pd
import numpy as np
import glob
import pyodbc
import time
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt

from db_connection import *
from googletrans import Translator
from deep_translator import GoogleTranslator
from nltk.corpus import stopwords
from tqdm import tqdm
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import feature_selection
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import make_pipeline, Pipeline
#from sklearn import pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, precision_recall_curve, confusion_matrix, roc_curve, auc
from sklearn.metrics import f1_score

import pickle

import warnings
warnings.filterwarnings("ignore")


## Define functions

In [2]:
def split_category(x):
    """ Split the category into different columns (max 9) """
    splitted = x.split('"')
    selected = list(map(splitted.__getitem__, list(range(1, min(18, len(splitted)), 2))))
    while len(selected) < 9:
        selected.append('N/A')
    return selected

In [3]:
def create_multiple_columns(df, column_name, prefix, drop_column=False):
    """ Create multiple columns from the given column containing lists """
    df_copy = df.copy()
    lists = df_copy[column_name].to_list()
    column_vals = list(zip(*lists))
    for idx, c in enumerate(column_vals):
        df_copy[f'{prefix}_{idx+1}'] = c
    if drop_column:
        df_copy.drop(column_name, axis=1, inplace=True)
    return df_copy

In [4]:
def transform_unicode(df, columns, values, replace_by):
    """ Replace tagged unicode with the replacement values in the designated column(s) """
    df_copy = df.copy()
    for v, r in zip(values, replace_by):
        for c in columns:
            df_copy[c] = df_copy[c].apply(lambda x: x.replace(v, r))
    return df_copy

In [5]:
def translate_title(df, to_language='nl', brand_translate=False, drop_titles=True):
    """ Translate the list of titles to the given language, keeping brands into account """
    # Ensure data copy and initialize looping variables and translation function
    df_copy = df.copy()
    translator = Translator(service_urls=['translate.googleapis.com'])
    time.sleep(2)
    titles, brands = list(df_copy['title']), list(df_copy['brand'])
    translated_titles = []

    for idx, title in tqdm(enumerate(titles)):
        
        # If brand is given, look to remove it from title before translation
        if str(brands[idx]) != '': #'nan'
            title_splitted, brand_splitted = title.split(' '), brands[idx].split(' ')
            cleaned_title, brand_idx = [], []
            for idx, word in enumerate(title_splitted):
                if word in brand_splitted:
                    brand_idx.append(idx)
                else:
                    cleaned_title.append(word)
            cleaned_title = ' '.join(cleaned_title)
            
            # Translated cleaned title and replace brand in title (order of words irrelevant)
            if brand_translate:
                translation = translator.translate(title, lang_tgt=to_language)
                # print(translation)
            else:
                translation = translator.translate(cleaned_title, lang_tgt=to_language)
                # print(translation)
            translation_splitted = translation.split(' ')
            translated_title = translation_splitted + brand_splitted
            translated_title = ' '.join(translated_title)
            
        # If brand is not given simply translate and return translation
        else:
            translated_title = translator.translate(title, lang_tgt=to_language)
    
        translated_titles.append(translated_title)
    
    # Insert data in dataframe and drop title if requested
    df_copy['title_nl'] = translated_titles 
    if drop_titles:
        df_copy.drop('title', axis=1, inplace=True)
    return df_copy   

In [7]:
# Strings to connect to a database
driver_server = 'host_server'   
db_name = 'etl_db'
user_name = 'etl_user_name' 
pasword = 'pa$$word'

# connect to a database
conn = connect_to_db(driver_server, db_name, user_name, pasword)
cursor = conn.cursor()

# select variables from a table in the given database
tables = pd.read_sql("select * from 'table_name';",conn)
tables

<h1> Data processing starts here </h1>

In [8]:
# copy the data frame
df = tables.copy()

In [9]:
# Disect category column and create separate columns for each (sub)category
df['category'] = df['category'].apply(lambda x: split_category(x))
df = create_multiple_columns(df=df, column_name='category', prefix='cat', drop_column=True)

In [10]:
# Transform unicode to string values
df = transform_unicode(df=df, 
                       columns=[col for col in df.columns if col[:3] == 'cat'], 
                       values=["\\u0026", "\\u0022"], 
                       replace_by=["&", " inch"])

In [11]:
# translate the descriptions of products
df = translate_title(df = df, 
                     to_language = 'nl', 
                     brand_translate = False, 
                     drop_titles = False)

In [12]:
# Alter to lower case
df["title_nl"] = df["title_nl"].str.lower()

In [13]:
# Remove product types with only few counts 
cats_list = ["Kindle Store", "Software", "Alexa Skills", "Digital Music", 
             "Home & Business Services", "Handmade", 
             "Magazine Subscriptions", "Gift Cards",
             "N/A", "Movies & TV", "CDs & Vinyl"]#, "Books", "Automotive"]
df = df.loc[~df.cat_1.isin(cats_list),:]

#### Add manually labeled data from the local digital market to enrich labelled data

In [14]:
# Retrieve local data
def localProductLineData(data_path):
    
    """Get manually validated (labelled) product lines from mollie shops"""
    
    df_new = pd.read_excel(data_path)
    df_new = df_new.loc[df_new.real_class != 'No Product',:]
    df_new = df_new.dropna(subset=['Correct'])
    display(df_new.Correct.value_counts())
    
    return df_new

data_dir = 'data_directory/data_name'
df_new = localProductLineData(data_dir)

In [15]:
# get labels and pick only columns that match with the variables from the database data
def get_Labels(data, x, y, z):
    
    for idx, row in data.iterrows():
        if row[y] == 1.0:
            data.loc[idx, 'cat_1'] = row[x]
        else:
            data.loc[idx, 'cat_1'] = row[z] 
    return data
df_new = get_Labels(df_new, 'predicted_class', 
                       'Correct', 'real_class')
df_new.rename(columns = {'Description': 'title_nl'}, 
                 inplace = True)
df_prodLine = df_new[['title_nl', 'cat_1']]

#### Get product lines datasets from a local directory that are related to (mono shops) shops that sell only a single product type

In [16]:
# specifying the path to excel files
path = "directory_for_files/*.xlsx"

def get_monoShops_prodline(path):
    
    # get files in the path
    list_of_files = glob.glob(path)

    # concatenate files
    files_list = []

    for file in list_of_files:
        files_list.append(pd.read_excel(file))

    # create an dataframe to store the concatenated data
    Merchant_merged = pd.concat(files_list, ignore_index=True)

    Merchant_merged.rename(columns={'Vertical':'cat_1',
                                        'Description': 'title_nl'}, 
                              inplace = True)
    
    Merchant_merged["cat_1"] = Merchant_merged["cat_1"].str.\
                replace('Automotive/Transportation', 'Transportation')
    
    return Merchant_merged

Merchant_prodline = get_monoShops_prodline(path)  

In [17]:
# merge all datasets imported above
df_amzMoll = pd.concat([df, df_prodLine, Merchant_prodline])
df_amzMoll["title_nl"] = df_amzMoll["title_nl"].str.lower()

In [18]:
# recode different names of the same label (class) for uniformity
df_amzMoll["cat_1"] = df_amzMoll["cat_1"].str.replace('Automotive', 'Transportation')

# remove classes that we dont need
df_amzMoll = df_amzMoll[df_amzMoll["cat_1"] != 'Erotics']

""" Remove numbers """
df_amzMoll["title_nl"] = df_amzMoll["title_nl"].str.replace('\d+','')
df_amzMoll = df_amzMoll.dropna(subset = ['title_nl'])
df_amzMoll.cat_1.value_counts()

In [19]:
df_amzMoll["title_nl"] = df_amzMoll["title_nl"].str.lower()

In [20]:
"""
The data considered is completely text data, which is the description (title) of the products purchased by consumers. Our machine learning model could not directly use the text data, however, with the help of NLP, we can transform it into a scaleable (numeric) value.  
To mention few of the actions to strike on, the removal of stopwords, lemmatization, Porter stemming, and the use of different algorithms, such as Term-Frequency Inverse Document Frequency (TF-IDF) are handy.
On the other hand, there is a feed-forward kind approach called the basic CountVectorizer that leads to acceptable results. 
It takes into account the entire words in the titles of products and counts them, and then assigns a number to each one based on its prevalence in the dataset, 
creating a bag of words matrix required by the model. The final step is to convert it to a dense array so it can be used by the Naive Bayes object.
"""

count_vec = CountVectorizer(max_features=150000)
bow = count_vec.fit_transform(df_amzMoll["title_nl"]) 
# vocablaries = count_vec.vocabulary_

In [21]:
# get association overview between labels and features
X_names = count_vec.get_feature_names_out()
y = df_amzMoll["cat_1"] 
p_value_limit = 0.95
df_features = pd.DataFrame()
for cat in np.unique(y):
    chis2, p = feature_selection.chi2(bow, y==cat)
    df_features = df_features.append(pd.DataFrame(
                   {"feature":X_names, "score":1-p, "y":cat}))
    df_features = df_features.sort_values(["y","score"], 
                    ascending=[True,False])
    df_features = df_features[df_features["score"] > p_value_limit]
X_names = df_features["feature"].unique().tolist()

In [22]:
for cat in np.unique(y):
    print("# {}:".format(cat))
    print("  . selected features:",
         len(df_features[df_features["y"]==cat]))
    print("  . top features:", ",".join(
            df_features[df_features["y"]==cat]["feature"].values[:10]))
    print(" ")

In [ ]:
# vectorizer = TfidfVectorizer(vocabulary=X_names)
# vectorizer.fit(df_amzMoll["title_nl"])
# X = vectorizer.transform(df_amzMoll["title_nl"])
# dic_vocabulary = vectorizer.vocabulary_

In [23]:
# Split data (train and test)
X_train, X_test, y_train, y_test = train_test_split(bow, y, stratify = y, test_size=0.2)

# classifier object
classifier = MultinomialNB()

# pipeline
model = Pipeline([("vectorizer", count_vec),  
                           ("classifier", classifier)])
# train classifier
model["classifier"].fit(X_train, y_train)

# make prediction
predicted = model["classifier"].predict(X_test)
predicted_prob = model["classifier"].predict_proba(X_test)

In [24]:
classes = np.unique(y_test)
y_test_array = pd.get_dummies(y_test, drop_first=False).values
    
## Accuracy, Precision, Recall
accuracy = accuracy_score(y_test, predicted)
auc = roc_auc_score(y_test, predicted_prob, 
                            multi_class="ovr")
print("Accuracy:",  round(accuracy,2))
print("Auc:", round(auc,2))
print("Detail:")
print(classification_report(y_test, predicted))
    
# Plot confusion matrix
cm = confusion_matrix(y_test, predicted)


fig, ax = plt.subplots(figsize=(25, 25))
sns.heatmap(cm, annot=True, fmt='d', ax=ax, cmap=plt.cm.Blues, 
            cbar=False)
ax.set(xlabel="Pred", ylabel="True", xticklabels=classes, 
       yticklabels=classes, title="Confusion matrix")
plt.yticks(rotation=0)
plt.xticks(rotation=90)

fig, ax = plt.subplots(nrows=1, ncols=2)

# Plot roc
for i in range(len(classes)):
    fpr, tpr, thresholds = roc_curve(y_test_array[:,i],  
                           predicted_prob[:,i])
    ax[0].plot(fpr, tpr, lw=3, 
              #label='{0} (area={1:0.2f})'.format(classes[i], 
              #                auc(fpr, tpr))
               )
ax[0].plot([0,1], [0,1], color='navy', lw=3, linestyle='--')
ax[0].set(xlim=[-0.05,1.0], ylim=[0.0,1.05], 
          xlabel='False Positive Rate', 
          ylabel="True Positive Rate (Recall)", 
          title="Receiver operating characteristic")
ax[0].legend(loc="lower right")
ax[0].grid(True)
    
## Plot precision-recall curve
for i in range(len(classes)):
    precision, recall, thresholds = precision_recall_curve(
                 y_test_array[:,i], predicted_prob[:,i])
    ax[1].plot(recall, precision, lw=3, 
               # label='{0} (area={1:0.2f})'.format(classes[i], 
               #                    auc(recall, precision))
              )
ax[1].set(xlim=[0.0,1.05], ylim=[0.0,1.05], xlabel='Recall', 
          ylabel="Precision", title="Precision-Recall curve")
ax[1].legend(loc="best")
ax[1].grid(True)
plt.show()


# Save the classifier as python object

In [25]:
with open("prod_cat_ml V20-12-2022.pkl", 'wb') as file:  
    pickle.dump(model, file)